In [1]:
import os
import numpy as np
import pandas as pd
from pandas import to_datetime
from matplotlib import pyplot as plt
import seaborn as sns
import itertools
import warnings
import datetime
from datetime import datetime
warnings.filterwarnings('ignore')

%matplotlib inline

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

import pmdarima as pm
from pmdarima.model_selection import train_test_split

In [2]:
w_list = [['temp', 'ph', 'ec', 'do', 'M73', 'toc', 'clola-a', 'rtwqi', 'ss', 'tn', 'tp', 'flow_rate', 'a_humidity', 'a_hpa', 'a_dew_point'], 
          ['temp', 'ph', 'ec', 'do', 'M73', 'toc', 'tn', 'tp', 'clola-a', 'rtwqi', 'a_humidity', 'a_hpa', 'a_dew_point'],
          ['temp', 'ph', 'ec', 'do', 'toc', 'tn', 'tp', 'clola-a', 'rtwqi', 'a_hpa', 'a_dew_point'], 
          ['temp', 'ph', 'ec', 'do', 'M73', 'toc', 'tn', 'tp', 'clola-a', 'rtwqi', 'w_depth', 'a_humidity', 'a_hpa', 'a_dew_point']]

op = 0

In [12]:
Data = pd.read_csv('../data/g_linear_amos_rtwqi0.csv')[:365*4*24]
Data['m_date'] = pd.to_datetime(Data['m_date'], format='%Y-%m-%d %H:%M', errors='raise')
Data.set_index('m_date', inplace=True)
Data = Data.resample('D').mean()
Data

,temp,ph,ec,do,M73,toc,clola-a,ss,tn,flow_rate,...,a_humidity,a_hpa,a_whpa,a_snow,a_dew_point,day_sin,day_cos,year_sin,year_cos,rtwqi
m_date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,8.431192,7.402167,382.357068,11.332912,5.535058,2.395833,7.479167,20.308866,4.384540,3.022691,...,49.875000,2.387500,1027.720833,0.0,-12.591667,3.830447e-02,2.308996e-02,-0.865729,0.987596,88.246923
2018-01-02,8.397057,7.399833,381.866588,11.329211,5.635836,2.393750,6.768750,1.600000,8.180000,2.530591,...,54.416667,2.387500,1028.958333,0.0,-13.025000,5.493401e-02,5.021074e-02,-0.701372,0.937095,88.300000
2018-01-03,8.341196,7.398028,382.480498,11.346740,5.761860,2.420833,5.387889,621.908247,9.445485,2.572326,...,36.916667,1.716667,1030.700000,0.0,-16.450000,5.532502e-02,5.123100e-02,-0.499607,0.848900,88.300000
2018-01-04,8.252522,7.388990,383.203618,11.329545,5.795508,2.433333,4.016667,640.906587,9.526712,2.653316,...,52.750000,2.137500,1028.395833,0.0,-13.670833,5.233683e-02,5.317434e-02,-0.298011,0.723037,88.213750
2018-01-05,8.256541,7.394382,382.290796,11.344389,5.929493,2.416667,4.979167,659.904927,9.607940,2.757008,...,60.291667,2.670833,1021.579167,0.0,-10.950000,4.700982e-02,5.548733e-02,-0.096645,0.559542,88.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-26,1.691667,7.877083,461.208333,13.854167,0.870833,1.956250,8.864324,1.482759,5.520103,4.779138,...,39.041667,1.033333,1035.362500,0.0,-22.216667,-8.956493e-14,7.389459e-14,-0.088149,0.996095,87.026250
2021-12-27,3.452595,7.770540,437.190790,13.504968,3.029097,2.118431,9.817293,1.500000,5.532000,4.705000,...,61.541667,2.066667,1032.937500,0.0,-14.854167,2.136208e-13,1.551213e-13,-0.071001,0.997464,85.721667
2021-12-28,1.047917,7.800000,463.916667,13.866667,0.700000,2.152083,9.114817,1.448276,5.545793,4.689310,...,80.666667,3.466667,1030.279167,0.0,-7.579167,6.738129e-14,-5.117897e-13,-0.053832,0.998538,85.656250


In [ ]:
auto_arima_model = pm.auto_arima(Data['temp'], seasonal=True, m=365)

In [ ]:
fcast = auto_arima_model.predict(30)

fig.ax = plt.subplots(figsize=(15,5))
chart = sns.lineplot(x='m_date', y='temp', data = df_train['temp'])
chart.set_title('ARIMA forecast - han river / temp')
fcast.plot(ax=ax, color='red', marker='o', legend=True)
